<a href="https://colab.research.google.com/github/sam1610/MIMiDataScience/blob/main/Projet_r%C3%A9seau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import numpy as np 
import copy
import pandas as pd 

In [260]:
class verif:
  def __init__(self, no_fic):
    self.fic=no_fic
    self.liste=[]

  def fic_to_liste(self, file):#Lecture ligne par ligne du fichier 
    l=[]
    for i in file:
      l.append(i)
    return l 

  def ouverture(self):
    try: 
      file=open(self.fic, "r")

      liste= self.fic_to_liste(file)#Liste representat le fichier passé en argument
          
      #Vérifier les offsets 
      b= self.verif_offset(liste) #Booleen true or false 
      if not(b):
        raise Exception("Erreur d'offset")
      else: #On a le bo offset 
        #Donc on enleve les commentaire de la liste 
        #Enlever les offsets vu qu'ils sont tous bon 
        liste= self.skip_offset(liste) #On a une liste sans les offsets 
            
        #Enlever les espaces 
        liste= self.enlever_espace(liste)
        print(liste)
        l= self.enlever_commentaires(liste)
        return l #Renvoyer la liste des octets 
    except: 
      display("Erreure ouverture du fichier")




    
  def enlever_espace(self, liste):
    l=[]
    for i in liste:
      l.append(i.replace(" ", "")) 
    return l 

  def verif_offset(self, liste):
    loff=[]
    for i in range(len(liste)):
      loff.append(int(liste[i][:4], 16))
    soff= loff.sort() #Sort les offset  
    if soff!=loff:
      False
    return  True 
  
  def skip_offset(self, l_originale):
    l=[]
    for i in l_originale:
      l.append(i[4:])
    return l 

  
  def enlever_commentaires(self, liste):
    l=[]
    for j in liste: 
      for i in range(0, len(j), 2):
        #pas de 2 pour recuperer les octets
        if i<len(j)-1:
          o= j[i]+j[i+1]
          #Verification que c'est un octet 
          if is_hex(o): 
            l.append(j[i]+j[i+1])
          else: 
            break 
    return l 

  def is_hex(self, oct):
    try: 
      h=int(oct, 16) #Convertir en decimale 
      return True #Valeure hexa 
    except: 
      return False #Valeure non hexa 


In [264]:

V= verif("My_tramcc.txt")
l= V.ouverture()
if V.ouverture()!=None:
  print("not opened")

l= ['f0189859ae320c8ddb1a1e8808004500...Y.2........E.\n', '01eae78d40002d06ca8c8077f50cc0a8....@.-....w....\n', '63c70050ed6ba3f0167dc08a8f988018c..P.k...}......\n', '00ebf20700000101080a945bf1fe4388...........[..C.\n', '9fa9485454502f312e312032..HTTP/1.1200O\n', '4b0d0a446174653a20467269K..Date:Fri,\n', '2053657020323032302030383a34383aSep202008:48:']

'Erreure ouverture du fichier'

'Erreure ouverture du fichier'

not opened


In [252]:
pwd

'/content/sample_data'

In [166]:
#Couche Ethernet

class Ethernet: 
  def __init__(self): 
    """Constructeur qui initialise les élements des la trame Etherne"""
    self.dest= "" 
    self.src="" 
    self.Et_type="" 

  def adresse_mac(self, trame, D_S):
    """Fonction qui retourne l'adresse Mac source ou destination d'une trame"""
    mac=""
    if D_S=="dest":  
      for i in range(6): 
        #Iterer sur les 6 octets pour récupérer l'adresse 
        if i!= 5: 
          #Affichage des : entre les octets 
          mac+= str(trame[i]) 
          mac+=":"
        else: 
          mac+= str(trame[i]) 
      self.dest=mac 
    else: 
      #il s'agit de l'adresse mac source 
      for i in range(6,12): 
        #Iterer sur les 6 octets pour récupérer l'adresse 
        if i!= 11: 
          #Affichage des : entre les octets 
          mac+= str(trame[i])
          mac+=":"
        else:
          mac+= str(trame[i]) 
      self.src=mac 

  def trame_type(self, trame):
    """Fonction qui trouve le type et sa difinition à partir d'un dictionnaire de types possibles"""
    Types={"0800":"IPv4","86dd": "Ipv6", "0805":"X.25","0806":"ARP","8035":"RARP"}
    t="" 
    for i in range(12,14): #Parcourir les 2 octets de type dans l'entete ethernet 
      t+=trame[i] #nous avons récupéré le type 
    if t in Types: #Trouvon s sa difinition 
      #Si le type dans la trame figure parmi les types du dictionnaire initialisé 
      return ("0x"+t, Types[t]) 
    else:
      return ("0x"+t, "Unknown type") 
    

In [132]:
#Test 

l= ["20", "52", "45", "43", "56", "00", "20", "53", "45", "4e", "44", "00", "08", "00", "45", "00", "00", "64", "12", "34", "00", "00", "ff", "11", "a3", "4f", "01", "01", "01", "01", "02", "02", "02", "02", '00', '00', '00', '00', '00', '50', '6b', 'fc', 'ff', 'ff' ] 

E= Ethernet()

E.adresse_mac(l, "dest")
E.adresse_mac(l, "src")
print(E.dest)
print(E.src)

E.trame_type(l)


20:52:45:43:56:00
20:53:45:4e:44:00


('0x0800', 'IPv4')

In [3]:
#Couche IP 

class IP(): 
  def __init__(self):
    self.version=""
    self.IHL=""
    self.tos="" 
    self.taille="" 
    self.id="" 
    self.flag={"R":"", "DF":"", "MF":""}
    self.fo="" 
    self.TTL=0
    self.protocole=""
    self.header_checksum=""
    self.ipsrc=""
    self.ipdest=""
    self.option_length=0 
    self.option="" 

  def determine_version(self, trame): 
    self.version= str(trame[14])[0]
    return self.version 

  def determine_ihl(self, trame):
     #ihl se trouve aprés le type de l'entete ethernet 
     oct= str(trame[14])
     self.IHL= int(oct[1], 16)
     return self.IHL


  def determine_tos(self, trame): 
    t= str(int(trame[15], 16)) 
    self.tos= t 
    return t 


  def total_length(self, trame):
    ll=""
    for i in range(16, 18):
      ll+=str(trame[i]) 
    self.taille= str(int(ll, 16)) 
    return self.taille  

  def determine_id(self, trame): 
    i=""
    for j in range(18, 20): 
      i+= trame[j]
    self.id= i
    return "0x"+i 

  def determine_flags(self, trame): 
    f= str(trame[20])[0] 
    f= bin(int(f, 16))[2:].zfill(len(f)*4) #ignonrer "0b" puis en utilisant zfill recuperer la valeure sur 4 bits 
    self.flag["R"]= f[0]
    self.flag["DF"]= f[1]
    self.flag["MF"]=  F[2]

  def analyse_flag(self, dflags): 
    ch="premier bit réservé à 0 \n"
    # Analyse DF 
    if dflagf["DF"]==0: 
      ch+="DF = 0 donc fragmentation possible\n"
    else: 
      #DF=1 
      ch+="DF=1 fonc fragmentation non autorisé\n"
    #ANALYSE MF
    if dflag["MF"]==0:
      ch+="MF=0 donc il ne y a pas de fragment suivant le fragment courant"
    else: 
      #MF=1 
      ch+="MF=1 donc il y a un fragment suivant le fragment courant"


  def determine_fragment_offset(self, trame): 
    f= "".join(trame[20:22]) #Recuperer le champ fragment offset de la trame 
    self.fo= bin(int(f, 16))[2:].zfill(len(f)*4)[3:] #Recuperer le 13 bits de poids faible appatenant au fragment offset car les 3 premiers appartiennent au flag 
    return self.fo
  
  def ttl(self, trame):
    t=str(trame[22])                       
    self.TTL= int(t,  16)  
    return self.TTL 

  def determine_protocole(self, trame): 
    prot= {"01": "ICMP", "06": "TCP", "11": "UDP"}
    #Recuperer le protocole 
    p= str(trame[23])
    if p in prot: 
      self.protocole= prot[p]
      return (p, prot[p])

  def determine_checksum(self, trame):
    t= "".join(trame[24:26])
    self.header_checksum=t
    return t 
    

  def ip_src(self, trame): 
    s=""
    for i in range(26, 30):
      if i!=29: 
        s+= str(trame[i])
        s+="."
      else: 
         s+= str(trame[i])
    self.ipsrc=s 
    return self.ipsrc
  
  def ip_dest(self, trame): 
    d=""
    for i in range(30, 34): 
      if i!=33: 
        s+= str(trame[i])
        s+="."
      else:
         s+= str(trame[i])
    self.ipdest=s 
    return self.ipdest     


  def determine_option(self, trame): 
    dtype={ "0": "EOOL", "1":"NOP", "7":"RR", "68":"TS", "131":"LSR", "137":"SSR"} #Options possibles en decimale 
    #Determiner le type: 
    t= int(trame[34], 16) #Convertir en decimale 
    if t in dtype: 
      self.option= dtype[t] 

  def determine_option_length(self, trame):
    ihl= self.determine_ihl(trame) 
    self.option_length= ( ihl*4 ) - 20 #20 octets sans options donc taille option = taille entete - 20 
    return self.option_length
  

In [4]:
i= IP()
print( "version ", i.determine_version(l))
print("ihl ", i.determine_ihl(l))
print("length", i.total_length(l))
print("id", i.determine_id(l))

version  4
ihl  5
length 100
id 0x1234


In [5]:
# Couche UDP

class UDP: 
  def __init__(self):
    self.ip=IP()
    self.sPort="" 
    self.dPort="" 
    self.length="" 
    self.checksum="" 
    self.trame_udp=[]

  def debut_udp(self, trame):
     ip_debut= trame[14:] #14 premiers octets appartiennent à ethernet
     l = self.ip.determine_option_length(trame) + 20
     self.trame_udp= ip_debut[l:] #La trame udp commence directement apres la fin des options de la trame ip 
     return self.trame_udp 

  def determine_source_port(self, trame):
    """ Fonction qui determine le port source d'une trame udp"""
    p= "".join(trame[0:2])
    self.sPort= p
    return int(p, 16) 
  
  def determine_dest_port(self, trame):
    """Fonction qui determine le port destination d'une trame udp"""
    p= "".join(trame[2:4])
    self.dport=p 
    return int(p, 16)  

  def determine_length(self, trame): 
    l="".join(trame[4:6])
    self.length= int(l, 16)
    return self.length

  def determine_checksum(self, trame): 
    c= "".join(trame[6:8])
    self.checksum= "0x"+c
    return self.checksum


In [6]:
u= UDP()
u.debut_udp(l)
udp= u.debut_udp(l)
print("source", u.determine_source_port(udp))
print("des", u.determine_dest_port(udp))
print("length", u.determine_length(udp))
print("checksum", u.determine_checksum(udp))

source 0
des 0
length 80
checksum 0x6bfc


In [115]:
#DNS
class DNS:

  def __init__(self):
    self.udp=UDP()
    self.trame_DNS=[]
    self.id=""
    self.flag="" 
    self.flagsdic={"Qr":"", "Opcode":"", "Aa":"", "Tc": "", "Rd":"", "Ra":"", "Z":"", "Rcode":""}
    self.no_questions= ""
    self.no_rep=""
    self.no_authority= ""
    self.no_add= ""
    self.questions=[] #Liste de toutes les questions 
    self.debut_answers=[] #Debut de la section des reponses dans la trame 
    self.answers=[]
    self.debut_authority=[] #Debut de la section authority dans la trame 
    self.authority=[]
    self.debut_additional=[] #Debut de la section additional dans la trame 
    self.additional=[]


  def debut_DNS(self, trame):
    trame_udp= self.UDP.debut_udp(trame) #La trame udp commence directement apres la fin des options de la trame ip 
    self.trame_DNS= trame_udp[8:]
    return self.trame_DNS 

  def determine_id(self, trame):
    self.id= "".join(trame[:2]) #deux premiers octets de la trame dns consacré au id
    return self.id

  def determine_flags(self, trame):
    flags="".join(trame[2:4]) 
    self.flag=flags
    binflags="" 
    for i in flags:
      binflags+=  bin(int(i, 16))[2:].zfill(len(i)*4)
    self.flagsdic["Qr"] = binflags[0]
    self.flagsdic["Opcode"] = binflags[1:5]
    self.flagsdic["Aa"] = binflags[5]
    self.flagsdic["Tc"] = binflags[6]
    self.flagsdic["Rd"] = binflags[7]
    self.flagsdic["Ra"] = binflags[8]
    self.flagsdic["Z"] = binflags[9]
    self.flagsdic["Rcode"] = binflags[10:14]
    return self.flagsdic 

  def determine_no_questions(self, trame):
    s=  "".join(trame[4:6])
    self.no_questions= int(s, 16)
    return self.no_questions
  
  def determine_no_rep(self, trame):
    s= "".join(trame[6:8])
    self.no_rep = int(s, 16)
    return self.no_rep

  def determine_no_authority(self, trame):
    s= "".join(trame[8:10])
    self.no_authority= int(s, 16)
    return self.no_authority 

  def determine_no_add(self, trame):
    s= "".join(trame[10:12])
    self.no_add= int(s, 16)
    return self.no_add 


  def compression_DNS(self, octet):
    """Une fonction qui renvoie le nombre d'octets a sauter pour recomencer la lecture"""
    binary="" 
    for i in octet: 
      binary+=bin(int(i, 16))[2:].zfill(len(i)*4) #Recuperer la valeure binaire sur 16 bits
    if binary[0]+binary[1] == '11': #On a bien un pointeur à cause des deu premiers bits 11 
      return(True, int(binary[2:], 2)) #On retourne le nombre d'octets a sauter
    else:
      return (False, 0) 

  def start_lecture(self, trameDns, nb):
    new = trameDns[nb:] #Retrouver le lieu où nous pouvons recommencer la lecture 

#Lire la trame octet par octet 
#et vérifier si on tombe sur un pointeur
#Si oui, appel de la fonction compression et trouver le nombre de saut necessaire 
#La condition d'arret va etre la lecture d'un 00 

  def hex_to_ascii(self, ch): 
    bytes_obj = bytes.fromhex(ch)
    return bytes_obj.decode('ASCII')


  def name_rec(self, tr, name, trame, c=[]):
    if tr[0]=="00":
      if c==[]: #on pas eu de pointeur 
        return name, tr
      else: 
        return name, c 
    #Sinon soit on est sur un caractere  normale (pas pointeur) soit on est sur un c0 (pointeur)
    (a, b)  = self.compression_DNS(tr[0]+tr[1])
    if a==False: #Pas un pointeur
      name+=tr[0] #On ajoute la valeure du code ascii
      if c!=[]: #Il s'agit d'une relecture 
        return self.name_rec(tr[1:], name, trame, c)
      else:
        return self.name_rec(tr[1:], name, trame) 
    else: #a=True 
      #On a un pointeur donc a=True on recupere b = nb octets a sauter
      print("tr", tr[2:])  
      return self.name_rec(trame[b:], name, trame, tr[2:])

  def determine_questions(self, trame, nbquestion): 
    tr= copy.deepcopy(trame[12:])
    b=0
    for i in range(nbquestion):
      n, liste= self.name_rec(tr[b:], "", trame)
      dic= {"Name":n, "Type":"".join(liste[0:2]), "Class":"".join(liste[2:4])}
      b=4
      tr=liste
      self.questions.append(dic)
    #idiquier où la fonction determine_answer doit commencer sa lecture 
    #Fini la lecture des questions, donc l'octet prochain est le debut de la lecture des reponses 
    self.debut_answers= tr[4:]
    return self.questions

  def hex_to_dec(self, lis): 
    return ".".join([str((int(i, 16))) for i in lis]) 

  def determine_answer(self, nbreponses, trame):
    tr= copy.deepcopy(self.debut_answers)
    b=0
    for i in range(nbreponses):
      n, liste= self.name_rec(tr[b:], "", trame)
      print("received", liste)
      dic= {"Name":n, "Type":"".join(liste[0:2]), "Class":"".join(liste[2:4]), "TTL": "".join(liste[4:8]), "DatalENGTH": "".join(liste[8:10]), "addr":self.hex_to_dec(liste[10:])}
      b=14
      tr=liste
      self.answers.append(dic)
    #idiquier où la fonction determine_answer doit commencer sa lecture 
    #Fini la lecture des questions, donc l'octet prochain est le debut de la lecture des reponses 
    self.debut_authority= tr[14:]
    return self.answers


  def determine_authority(self, nbauthority, trame):
    tr= copy.deepcopy(self.debut_authority)
    b=0
    for i in range(nbauthority):
      n, liste= self.name_rec(tr[b:], "", trame)
      dic= {"Name":n, "Type":"".join(liste[0:2]), "Class":"".join(liste[2:4]), "TTL": "".join(liste[4:8]), "DatalENGTH": "".join(liste[8:10]), "addr":self.hex_to_dec(liste[10:])}
      b=14
      tr=liste
      self.authority.append(dic)
    #idiquier où la fonction determine_answer doit commencer sa lecture 
    #Fini la lecture des questions, donc l'octet prochain est le debut de la lecture des reponses 
    self.debut_additional= tr[14:]
    return self.authority 

  def determine_additional(self, nbadd, trame):
    tr= copy.deepcopy(self.debut_additional)
    b=0
    for i in range(nbadd):
      n, liste= self.name_rec(tr[b:], "", trame)
      dic= {"Name":n, "Type":"".join(liste[0:2]), "Class":"".join(liste[2:4]), "TTL": "".join(liste[4:8]), "DatalENGTH": "".join(liste[8:10]), "addr":self.hex_to_dec(liste[10:])}
      b=14
      tr=liste
      self.additional.append(dic)
    #idiquier où la fonction determine_answer doit commencer sa lecture 
    #Fini la lecture des questions, donc l'octet prochain est le debut de la lecture des reponses 
    self.debut_additional= tr[14:]
    return self.additional
  

In [117]:
# Test
dns= DNS()
# trame= ["35", "56", "01", "00", "00", "01", "00", "00", "00", "00", "00", "00", "05", "66", "6f", "6e", "74", "73", "0a", "67", "6f", "6f", "67", "6c", "65", "61", "70", "69", "73", "03", "63", "6f", "6d", "00", "00", "01", "00", "01"] 


trame= ["56", "3c", "81", "80", "00", "01", "00", "01", "00", "00", "00", "00", "0e", "65", "6e", "63", "72", "79", "70", "74", "65", "64", "2d", "74", "62", "6e", "30", "07", "67", "73", "74","61", "74", "69", "63", "03", "63", "6f", "6d", "00", "01", "00", "01", "c0", "0c","00", "01", "00", "01", "00", "00", "00","a3", "00", "04", "d8", "3a", "c9", "ee" ]
print("id", dns.determine_id(trame)) 
print( "flags", dns.determine_flags(trame))
print("valeure flag", dns.flag)

print("no questions", dns.determine_no_questions(trame))
print("no answers", dns.determine_no_rep(trame)) 
print("no Authority", dns.determine_no_authority(trame))
print("no addiotional", dns.determine_no_add(trame))
print("questions", dns.determine_questions(trame, 1))
print("Answers", dns.determine_answer( 1, trame))

print("authority", dns.determine_authority(0,trame))
print("Additional", dns.determine_additional(0, trame))

# dns.compression_DNS("C00C")
print()



id 563c
flags {'Qr': '1', 'Opcode': '0000', 'Aa': '0', 'Tc': '0', 'Rd': '1', 'Ra': '1', 'Z': '0', 'Rcode': '0000'}
valeure flag 8180
no questions 1
no answers 1
no Authority 0
no addiotional 0
questions [{'Name': '0e656e637279707465642d74626e30076773746174696303636f6d', 'Type': '0001', 'Class': '0001'}]
tr ['00', '01', '00', '01', '00', '00', '00', 'a3', '00', '04', 'd8', '3a', 'c9', 'ee']
received ['00', '01', '00', '01', '00', '00', '00', 'a3', '00', '04', 'd8', '3a', 'c9', 'ee']
Answers [{'Name': '0e656e637279707465642d74626e30076773746174696303636f6d', 'Type': '0001', 'Class': '0001', 'TTL': '000000a3', 'DatalENGTH': '0004', 'addr': '216.58.201.238'}]
authority []
Additional []



In [ ]:
#DHCP
 
class DHCP:

  def __init__(self):
    self.op="" 
    self.htypeval="" 
    self.htype="" #A determiner parmi plueisurs 
    self.hlen=0 
    self.hops="" 
    self.xid="" 
    self.secs=0 
    self.flags= {"R":"", "B":""} 
    self.ciaddr="" 
    self.yiaddr="" 
    self.siaddr="" 
    self.giaddr="" 
    self.chaddr="" 
    self.sname="" 
    self.file="" 
    self.options="" 


  def determine_op(self, trame): 
    """l'argument trame est une trame dhcp"""
    self.op= trame[0]
    return self.op 

  def determine_htypeVal(self, trame):
    v= "".join(trame[1:3]) 
    self.htypeval= int(v, 16)
    return self.htypeval


  def determine_htype(self, htypeval):
    if htypeval==1: 
      self.htype="Ethernet"
    elif htypeval==6:
      self.htype="IEE 802 Networks"
    elif htypeval==7:
      self.htype="ARCNET"
    elif htypeval==11:
      self.htype="LocaITalk"
    elif htypeval==12:
      self.htype="LocaINet (IBM PCNet or SYTEK LocaINET)"
    elif htypeval==14:
      self.htype="SMDS"
    elif htypeval==15:
      self.htype="Frame Relay"
    elif htypeval==16:
      self.htype="Asynchronous Transfer mode (ATM)"
    elif htypeval==17:
      self.htype="HDLC"
    elif htypeval==18:
      self.htype="Fibre Channel"
    elif htypeval==19:
      self.htype="Asynchronous Transfer Mode (ATM)"
    elif htypeval==20:
      self.htype="Serial Line"
    
    return self.htype


  def determine_hlen(self, trame):
    val= "".join(trame[3:6])
    self.hlen= int(val, 16) #Dec 
    return self.hlen 


  def determine_hops(self, trame):
    self.hops= "".join(trame[6: 10])
    return self.hops

  def determine_xid(self, trame): 
    val="".join(trame[10:20])
    self.xid=int(val, 16)
    return self.xid

  def determine_secs(self, trame): 
    val="".join(trame[20:23])
    self.secs=nt(val, 16)
    return self.secs


  def determine_flags(self, trame): 
    v= bin(int("".join(trame[23], 16))) 
    self.flags["B"] = v[2]
    self.flags["R"] = bin(int("".join(trame[24:30]), 16))[2:]
    self.flags["R"] = v[3:] + self.flags["R"]
    

  def determine_ciaddr(self, trame):
    self.ciaddr=".".join(trame[30:40])
    return self.ciaddr

  
  def determine_yiaddr(self, trame):
    self.yiaddr=".".join(trame[40:50])
    return self.yiaddr

  def determine_siaddr(self, trame):
    self.siaddr=".".join(trame[50:60])
    return self.siaddr

  def determine_giaddr(self, trame):
    self.giaddr=".".join(trame[60:70])
    return self.giaddr

  def determine_chaddr(self, trame):
    self.chaddr="".join(trame[70:80])
    return self.chaddr

  def determine_sname(self, trame):
    self.sname="<"
    self.sname+="".join(trame[80:90])
    self.sname+=">"
    return self.sname

  
  def determine_file(self, trame):
    self.file="<"
    self.file+="".join(trame[90:100])
    self.file+=">"
    return self.file

  def determine_optionval(self, trame):
    val= trame[100] #100eme octet correspond au code de l'option 
    return val 

  def determine_option(self, trame): 
    v= self.determine_optionval(trame) 
    v= int(v, 16) #Recuperer la valeure de l'option en decimale 
    l_op= [1, 3, 15, 6, 12, 28, 43, 50, 51, 53, 54, 55, 57, 58, 59, 60, 61, 114, 116, 2, 224]
    if v in l_op: 
      if v==1: 
        self.option= "Subnet Mask"
      elif v==3: 
        self.option= "Router"
      elif v==15: 
        self.option= "Domain Name"
      elif v==6:
        self.option="Serveur de domaine"
      elif v==12:
        self.option="Nom d'hôte"
      elif v==28:
        self.option="Adresse de diffusion"
      elif v==43:
        self.option="Spécifique au fournisseur"
      elif v==50:
        self.option="Demande d'adresse"
      elif v==51:
        self.option="Adresse Heure"
      elif v==53:
        self.option="Type de message DHCP"
      elif v==54:
        self.option="Identifiant du serveur DHCP"
      elif v==55:
        self.option="Liste des paramètres"
      elif v==57:
        self.option="Taille maximale des messages DHCP"
      elif v==58:
        self.option="Temps de renouvellement"
      elif v==59:
        self.option="Temps de reliure"
      elif v==60:
        self.option="Identifiant de classe"
      elif v==61:
        self.option="Identité du client"
      elif v==114:
        self.option="Portail captif DHCP"
      elif v==116:
        self.option="Configuration automatique"
      elif v==2:
        self.option="Décalage horaire"
      elif v==224:
        self.option="Réservé (usage privé)"
    else: 
      self.option= "Option non traité par le programme"

    return self.option 



In [267]:
# import Ethernet as eth
# import IP.py 
# import UDP.py 
# import DNS.py
# import DHCP.py 
# import Verif as ver 
import argparse 

#Recuperer le fichier source et destination 

#Instentiation de la classe ArgumentParser 
Args = argparse.ArgumentParser()

#Ajouter les arguments nécessaires
Args.add_argument("fic_src",type=str,help="Veuillez saisir le chemin vers le fichier contenant la trame\n")
Args.add_argument("fic_dest",type=str,help="Veuillez saisir le nom du fichier qui contiendra les analyses de la trame\n")
 
if ver.verif(fic_src)!=None: #Nous avons reussit a ouvrir le fichier! 
  #On ouvre le fichier destination 
  f= (fic_des, "a") #Ouvrir le fichier en mode append 
  #On commence les analyses par Ethernet 
  f.write("__________________________________________Ethernet_________________________________________")







# --------------------------Ethernet-----------------------
# source 'des
# type
# ------------------------IP-------------------------------
# ihl
# version
# option
# flag/
# -----------UDP--------------------

#Recuperer 

NameError: ignored

In [ ]:
L=["01", "01", "06", "00", "50", "79", "09","ab",  ]

In [ ]:
l=["1", "2", "aa", "bb"]
".".join(l[1:4])

'2.aa.bb'

In [ ]:
str(str(int("07", 16)))
print("this IS\n IT")

this IS
 IT


In [ ]:
chr(int("97"))

'a'

In [62]:
l=[1, 2, 3, 4, 5]


def f(l):
  for i in range(2):
    l=l[1:]


f(l)
print(l)

[1, 2, 3, 4, 5]


In [32]:
chr(int('0e', 16))

'\x0e'

In [43]:
n= int('0e', 16)
print(n)
binascii.unhexlify('%x' % 14) 

14


Error: ignored

In [69]:
q=0
for i in range(2):
  q+=1

print(q)

2


In [107]:
l=["d8", "3a", "c9", "ee"]
".".join([str((int(i, 16))) for i in l]) 

'216.58.201.238'

In [120]:
hex_str = '0e656e637279707465642d74626e30076773746174696303636f6d'

bytes_obj = bytes.fromhex(hex_str)

ascii_str = bytes_obj.decode('utf-8')

print(ascii_str)

encrypted-tbn0gstaticcom


In [180]:
l=[1, 2, 3]
l1=[2, 3, 1]
l==l1

False